In [3]:
!pip install -q tqdm gensim

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
import pandas as pd
import glob
import json
from tqdm import tqdm
import re
from datetime import datetime
import gensim
import logging
import os
import shutil
import pprint

In [5]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [6]:
df = pd.read_csv("talks.csv.gzip", compression="gzip")

In [10]:
df.head()

,Unnamed: 0,average_rating,categories,description,dislike_count,duration,id,like_count,thumbnail,title,upload_date,view_count,webpage_url,transcript
0,0,4.729469,['Nonprofits & Activism'],http://www.ted.com When German psychologist In...,14.0,830,Jc2F3-nawnI,193.0,https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg,Inge Missmahl brings peace to the minds of Afg...,20100929,20695,https://www.youtube.com/watch?v=Jc2F3-nawnI,So I want to tell you a story -- an encouragin...
1,1,4.940791,['News & Politics'],"As a child growing up in North Korea, Hyeonseo...",2151.0,735,PdxPCeWw75k,143165.0,https://i.ytimg.com/vi/PdxPCeWw75k/maxresdefau...,My escape from North Korea | Hyeonseo Lee,20130320,6702386,https://www.youtube.com/watch?v=PdxPCeWw75k,Translator: Joseph Geni\nReviewer: Morton Bast...
2,2,4.554828,['News & Politics'],"http://www.ted.com Every day, in a city the si...",68.0,938,CLWRclarri0,543.0,https://i.ytimg.com/vi/CLWRclarri0/hqdefault.jpg,Carolyn Steel: How food shapes our cities,20091005,125975,https://www.youtube.com/watch?v=CLWRclarri0,How do you feed a city?\nIt's one of the great...
3,3,4.706122,['Science & Technology'],http://www.ted.com With his team at SENSEable ...,54.0,947,CijsvAGU6-c,681.0,https://i.ytimg.com/vi/CijsvAGU6-c/hqdefault.jpg,Carlo Ratti: Architecture that senses and resp...,20110503,78537,https://www.youtube.com/watch?v=CijsvAGU6-c,"Good afternoon, everybody.\nI've got something..."
4,4,4.732777,['Travel & Events'],"Almost everything we own and use, at some poin...",96.0,684,j7RsRnYlz7I,1341.0,https://i.ytimg.com/vi/j7RsRnYlz7I/maxresdefau...,Rose George: Inside the secret shipping industry,20131213,144699,https://www.youtube.com/watch?v=j7RsRnYlz7I,"A couple of years ago,\nHarvard Business Schoo..."


In [25]:
pprint.pprint(df.iloc[0].to_dict())

{'Unnamed: 0': 0,
 'average_rating': 4.72946882248,
 'categories': "['Nonprofits & Activism']",
 'description': 'http://www.ted.com When German psychologist Inge Missmahl '
                'went to Afghanistan, she saw universal wounds of the human '
                'heart -- despair and trauma. She tackled this widespread '
                'depression with psychosocial counseling, and in return '
                'witnessed remarkable individual and social healing and new '
                'hope for families and communities.\n'
                '\n'
                'TEDTalks is a daily video podcast of the best talks and '
                "performances from the TED Conference, where the world's "
                'leading thinkers and doers give the talk of their lives in 18 '
                'minutes. Featured speakers have included Al Gore on climate '
                'change, Philippe Starck on design, Jill Bolte Taylor on '
                'observing her own stroke, Nicholas Negropon

In [23]:
print(df.iloc[0].to_dict()['description'])

http://www.ted.com When German psychologist Inge Missmahl went to Afghanistan, she saw universal wounds of the human heart -- despair and trauma. She tackled this widespread depression with psychosocial counseling, and in return witnessed remarkable individual and social healing and new hope for families and communities.

TEDTalks is a daily video podcast of the best talks and performances from the TED Conference, where the world's leading thinkers and doers give the talk of their lives in 18 minutes. Featured speakers have included Al Gore on climate change, Philippe Starck on design, Jill Bolte Taylor on observing her own stroke, Nicholas Negroponte on One Laptop per Child, Jane Goodall on chimpanzees, Bill Gates on malaria and mosquitoes, Pattie Maes on the "Sixth Sense" wearable tech, and "Lost" producer JJ Abrams on the allure of mystery. TED stands for Technology, Entertainment, Design, and TEDTalks cover these topics as well as science, business, development and the arts. Closed

In [26]:
print(df.iloc[0].to_dict()['transcript'])

So I want to tell you a story -- an encouraging story --
about addressing
desperation, depression and despair in Afghanistan,
and what we have learned from it,
and how to help people
to overcome traumatic experiences
and how to help them to regain some confidence
in the time ahead -- in the future --
and how to participate again in everyday life.
So, I am a Jungian psychoanalyst,
and I went to Afghanistan in January 2004, by chance,
on an assignment for Medica Mondiale.
Jung in Afghanistan --
you get the picture.
Afghanistan is one of the poorest countries in the world,
and 70 percent of the people are illiterate.
War and malnutrition kills people
together with hope.
You may know this from the media,
but what you may not know
is that the average age of the Afghan people is 17 years old,
which means they grow up in such an environment
and -- I repeat myself --
in 30 years of war.
So this translates
into ongoing violence,
foreign interests, bribery,
drugs, ethnic conflicts,
bad health, s

In [7]:
datetime.strptime('20100929', '%Y%m%d')


datetime.datetime(2010, 9, 29, 0, 0)

In [8]:
df.head()

,Unnamed: 0,average_rating,categories,description,dislike_count,duration,id,like_count,thumbnail,title,upload_date,view_count,webpage_url,transcript
0,0,4.729469,['Nonprofits & Activism'],http://www.ted.com When German psychologist In...,14.0,830,Jc2F3-nawnI,193.0,https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg,Inge Missmahl brings peace to the minds of Afg...,20100929,20695,https://www.youtube.com/watch?v=Jc2F3-nawnI,So I want to tell you a story -- an encouragin...
1,1,4.940791,['News & Politics'],"As a child growing up in North Korea, Hyeonseo...",2151.0,735,PdxPCeWw75k,143165.0,https://i.ytimg.com/vi/PdxPCeWw75k/maxresdefau...,My escape from North Korea | Hyeonseo Lee,20130320,6702386,https://www.youtube.com/watch?v=PdxPCeWw75k,Translator: Joseph Geni\nReviewer: Morton Bast...
2,2,4.554828,['News & Politics'],"http://www.ted.com Every day, in a city the si...",68.0,938,CLWRclarri0,543.0,https://i.ytimg.com/vi/CLWRclarri0/hqdefault.jpg,Carolyn Steel: How food shapes our cities,20091005,125975,https://www.youtube.com/watch?v=CLWRclarri0,How do you feed a city?\nIt's one of the great...
3,3,4.706122,['Science & Technology'],http://www.ted.com With his team at SENSEable ...,54.0,947,CijsvAGU6-c,681.0,https://i.ytimg.com/vi/CijsvAGU6-c/hqdefault.jpg,Carlo Ratti: Architecture that senses and resp...,20110503,78537,https://www.youtube.com/watch?v=CijsvAGU6-c,"Good afternoon, everybody.\nI've got something..."
4,4,4.732777,['Travel & Events'],"Almost everything we own and use, at some poin...",96.0,684,j7RsRnYlz7I,1341.0,https://i.ytimg.com/vi/j7RsRnYlz7I/maxresdefau...,Rose George: Inside the secret shipping industry,20131213,144699,https://www.youtube.com/watch?v=j7RsRnYlz7I,"A couple of years ago,\nHarvard Business Schoo..."


In [27]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [ ]:
?text.TfidfVectorizer

In [28]:
from sklearn.feature_extraction import text
Text=df['transcript'].tolist()
tfidf=text.TfidfVectorizer(input=Text,stop_words="english", tokenizer=LemmaTokenizer())
matrix=tfidf.fit_transform(Text)
#print(matrix.shape)

In [29]:
### Get Similarity Scores using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
sim_unigram=cosine_similarity(matrix)

In [30]:
sim_unigram

array([[1.        , 0.42328378, 0.55359966, ..., 0.49388626, 0.53364983,
        0.58179026],
       [0.42328378, 1.        , 0.4507271 , ..., 0.43143527, 0.42768029,
        0.45480349],
       [0.55359966, 0.4507271 , 1.        , ..., 0.55640022, 0.57594978,
        0.61223904],
       ...,
       [0.49388626, 0.43143527, 0.55640022, ..., 1.        , 0.51827279,
        0.5542238 ],
       [0.53364983, 0.42768029, 0.57594978, ..., 0.51827279, 1.        ,
        0.56661015],
       [0.58179026, 0.45480349, 0.61223904, ..., 0.5542238 , 0.56661015,
        1.        ]])

In [31]:
def get_similar_articles(x):
    return "\n".join(df['title'].loc[x.argsort()[-5:-1]])
df['similar_articles_unigram']=[get_similar_articles(x) for x in sim_unigram]

In [32]:
df['title'][1]

'My escape from North Korea | Hyeonseo Lee'

In [ ]:
print (df['similar_articles_unigram'][1])

In [ ]:
# Problems with this model

# Doc2Vec

In [33]:
df.iloc[0]

Unnamed: 0                                                                  0
average_rating                                                        4.72947
categories                                          ['Nonprofits & Activism']
description                 http://www.ted.com When German psychologist In...
dislike_count                                                              14
duration                                                                  830
id                                                                Jc2F3-nawnI
like_count                                                                193
thumbnail                    https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg
title                       Inge Missmahl brings peace to the minds of Afg...
upload_date                                                          20100929
view_count                                                              20695
webpage_url                       https://www.youtube.com/watch?

In [34]:
[row["title"]] + row["categories"]

NameError: name 'row' is not defined

In [ ]:
talk_corpus = []
for (index,row) in df.iterrows():
    content = gensim.utils.simple_preprocess (row["transcript"])
    #tags = [row["title"], row["id"]] + row["categories"]'
    #tags = [row["title"]] + row["categories"]
    tags = [row["title"]] 
    tg = gensim.models.doc2vec.TaggedDocument(content,tags)
    talk_corpus.append(tg)

In [ ]:
talk_corpus[0]

In [ ]:
model = gensim.models.Doc2Vec(size = 300, 
                              min_count = 3, 
                              iter = 100)

In [ ]:
model.build_vocab(talk_corpus)
print("model's vocabulary length:", len(model.wv.vocab))

In [ ]:
model.train(talk_corpus,total_examples=model.corpus_count,epochs=model.epochs)

In [ ]:
model.docvecs.most_similar('My escape from North Korea | Hyeonseo Lee')

In [ ]:
df.head(1)

In [ ]:
content = gensim.utils.simple_preprocess (df.iloc[0]["transcript"])

In [ ]:
content

In [ ]:
new_vector = model.infer_vector(content)
sims = model.docvecs.most_similar([new_vector],topn=20)
sims